# Foursquare Star

### Introduction

In this lesson, we'll practice working with our foursquare domain to perform ETL in postgres.  We'll do this to copy our data from a schema that fits OLTP to one that fits a OLAP structure.  

### Loading the Data

Skip this section if you already have a foursquare database in postgres, otherwise, follow the following steps.

* Create a database called `foursquare_practice`.
* Do so from the command line with the -c flag.


Then run the migrations in the `db/migrations/create_tables.sql` file.

Now that our database and related tables are created, let's load in some data. We have a number of CSV files in the `foursquare-fullstack/data` folder. We can copy in the data into our tables with a command in the following format:
```SQL
COPY table_name
FROM 'absolute/path/to/data.csv' 
DELIMITER ',' 
CSV HEADER;
```
Load in each csv file in the data folder to the related table.

> Note: Begin with the least dependent tables (eg. states then cities)

### Reviewing Foursquare

Once our database schema is setup and our data is loaded, let's connect to our foursquare database, and take a look at the various tables in our OLTP schema.  Our starting foursquare schema looks like the following.

> <img src="./oltp_venue.png" width="80%">

And we'll move this data to a star schema that takes on the following structure.

> <img src="./updated_star.png" width="50%">

So above we have a fact table of venues, and dimension tables of locations and categories.

To get there, the main steps that we'll need to perform is to select the data spread across zipcodes, cities and states tables in our OLTP structure and then move it into the `locations` table.  

### Creating our Tables

Ok, our first step is to create our new tables.  To that end we have created a `star_schema.sql` file located in the root directory of the lab.  

Take a look through the file, and then run the migrations using the `-f` flag against the `foursquare_practice` database.

Let's take a look at one of the tables, `dimcategories`:

```SQL
CREATE TABLE IF NOT EXISTS dimcategories (
  id serial PRIMARY KEY,
  name VARCHAR(255),
  venue_id INTEGER,
  CONSTRAINT fk_venue
  FOREIGN KEY (venue_id)
  REFERENCES factvenues (id)
);
```

So one of the key changes here is that we no longer have a constraint that a category's `name` be unique.  The reason is because now that we are moving away from third normal form (3NF), we will repeat categories like`Pizza Place` for every pizza venue, and simply have a different venue_id for each record.  This means we can no longer use the unique constraint. 

### Copying Data

Ok, now let's work on copying over our data.  Let's start with the dimension table of locations. 

> We do this because it is the only table that does not have a foreign key.

We'll do this in two steps.  First, we'll make sure we are selecting the correct data by only select the correct data and not inserting it.  Then after we know we are selecting the right data, we can move it to the right table with an  `INSERT INTO..SELECT` command.

> First, we connect to the database.

In [80]:
import psycopg2
def get_cursor():
    conn = psycopg2.connect(
    host="127.0.0.1",
    database="foursquare_development",
    user="postgres",
    password="postgres")
    
    cursor = conn.cursor()
    return conn, cursor

In [81]:
conn, cursor = get_cursor()

> And now simply select (without inserting in) the data needed for the `dimlocations` table and limit to two rows of data.  Also select the location id value.

In [31]:
cursor.execute("""SELECT locations.id, latitude, longitude, code, cities.name, states.name 
FROM locations 
JOIN zipcodes ON locations.zipcode_id = zipcodes.id
JOIN cities ON zipcodes.city_id = cities.id
JOIN states ON cities.state_id = states.id LIMIT 2;
""")

In [32]:
cursor.fetchall()
# [(50, Decimal('-73.9875'), Decimal('40.7024'), 11210, 'New York', 'New York'),
#  (51, Decimal('-73.9875'), Decimal('40.7024'), 11210, 'New York', 'New York')]

[(50, Decimal('-73.9875'), Decimal('40.7024'), 11210, 'New York', 'New York'),
 (51, Decimal('-73.9875'), Decimal('40.7024'), 11210, 'New York', 'New York')]

Ok, now that we have the `SELECT` component working, we can add in our `INSERT INTO` statement.  We'll have the id be equal to our original locations id in the OLTP.  This will help us find the original data as needed.

In [57]:
cursor.execute("""
INSERT INTO dimlocations (id, longitude, latitude, address, zipcode, city, state)
SELECT locations.id, longitude, latitude, address, code, cities.name, states.name 
FROM locations 
JOIN zipcodes ON locations.zipcode_id = zipcodes.id
JOIN cities ON zipcodes.city_id = cities.id
JOIN states ON cities.state_id = states.id;
""")

Then we'll need to commit the change.

In [ ]:
conn.commit()

And we can confirm that our command worked.

In [52]:
cursor.execute('SELECT * FROM dimlocations LIMIT 1;')

In [53]:
cursor.fetchall()
# [(50, Decimal('40.7024'), Decimal('-73.9875'), '141 Front Street', '11210', 'New York', 'New York')]

[(50,
  Decimal('40.7024'),
  Decimal('-73.9875'),
  '141 Front Street',
  '11210',
  'New York',
  'New York')]

In [59]:
cursor.execute('SELECT COUNT(*) FROM dimlocations LIMIT 1;')
cursor.fetchall()

[(134,)]

Ok, now let's take care of the `factvenues` table.  We'll need the `id`, `name`, `location_id`, `price`, `rating`, and `likes`.  The `id` should match the venue id of the venues table, and the `location_id` can match the `location_id` from the original OLTP schema.

When you feel comfortable, insert the data into factvenues table.

In [68]:
conn, cursor = get_cursor()

cursor.execute("""
INSERT INTO factvenues (id, name, location_id, price, rating, likes)
SELECT venues.id, venues.name, locations.id, price, rating, likes FROM venues
JOIN locations ON venues.id = locations.venue_id;
""")

conn.commit()

In [69]:
cursor.execute("SELECT * FROM factvenues LIMIT 5;")

In [70]:
cursor.fetchall()
# [(53, 'Los Tacos Al Pastor', 50, 1, None, None),
#  (54, 'Grimaldis', 51, 2, Decimal('2'), 3),
#  (55, 'Cafe Mogador', 52, 3, Decimal('4'), 15),
#  (56, 'Zahavs', 53, 4, Decimal('5'), 100),
#  (57, 'Los Tacos Al Pastor', 54, 1, Decimal('8.0'), 52)]

[(53, 'Los Tacos Al Pastor', 50, 1, None, None),
 (54, 'Grimaldis', 51, 2, Decimal('2'), 3),
 (55, 'Cafe Mogador', 52, 3, Decimal('4'), 15),
 (56, 'Zahavs', 53, 4, Decimal('5'), 100),
 (57, 'Los Tacos Al Pastor', 54, 1, Decimal('8.0'), 52)]

And the last step is our `dimcategories` table.  Here, we'll need to insert in the `venue_id`, and category `name`.  Again, it's beneficial to begin with the SELECT statement and then move to the INSERT INTO statement.

> Here, we can set the id equal to the `venue_categories.id`.

In [84]:
cursor.execute("""
INSERT INTO dimcategories (id, venue_id, name)
SELECT venue_categories.id, venue_id, categories.name FROM venue_categories 
JOIN categories ON categories.id = venue_categories.category_id LIMIT 4;
""")

In [85]:
conn.commit()

In [86]:
cursor.execute("SELECT * FROM dimcategories LIMIT 2;")

In [87]:
cursor.fetchall()
# [(68, 'Pizza', 54), (69, 'Italian', 54)]

[(68, 'Pizza', 54), (69, 'Italian', 54)]

### Summary

In this lesson, we practiced migrating our data from the OLTP schema to the OLAP schema.  We broke this into two steps: we first made sure that we had the `SELECT` statement working properly, and then we added in the `INSERT INTO` component.